In [3]:
import numpy as np
import random

# 활성화 함수 - 1. 시그모이드
def sigmoid(x, derivative=False):
    if (derivative==True):
        return x * (1-x)
    return 1 / (1 + np.exp(-x))

# 활성화 함수 - 2. tanh
def tanh(x, derivative=False):
    if (derivative==True):
        return 1 - np.tanh(x) ** 2
    return np.tanh(x)

# 가중치 배열을 만드는 함수
def makeMatrix(i, j, fill=0.0):
    mat = []
    for i in range(i):
        mat.append([fill] * j)
    return mat

In [2]:
data = [
    [[0, 0], [0]],
    [[0, 1], [1]],
    [[1, 0], [1]],
    [[1, 1], [0]]
]

# 실행 횟수(iterations), 학습률(lr), 모멘텁 계수(mo) 설정
itertations = 5000
lr = 0.1
mo = 0.9


In [6]:
class NeuralNetwork:
    def __init__(self, num_x, num_yh, num_yo, bias=1):
        # 입력 값(num_x), 은닉층의 초기값(num_yh), 출력층의 초기값(num_yo), bias
        self.num_x = num_x + bias
        self.num_yh = num_yh
        self.num_yo = num_yo

        # 활성화 함수 초기값
        self.activation_input = [1.0] * self.num_x
        self.activation_hidden = [1.0] * self.num_yh
        self.activation_out = [1.0] * self.num_yo

        # 가중치 입력 초기값
        self.weight_in = makeMatrix(self.num_x, self.num_yh)
        for i in range(self.num_x):
            for j in range(self.num_yh):
                self.weight_in[i][j] = random.random()

        # 가중치 출력 초기값
        self.weight_out = makeMatrix(self.num_yh, self.num_yo)
        for i in range(self.num_yh):
            for j in range(self.num_yo):
                self.weight_out[i][j] = random.random()

        self.gradient_in = makeMatrix(self.num_x, self.num_yh)
        self.gradient_out = makeMatrix(self.num_yh, self.num_yo)

    def update(self, inputs):
        # 입력층의 활성화 함수
        for i in range(self.num_x - 1):
            self.activation_input[i] = inputs[i]

        # 은닉층의 활성화 함수
        for j in range(self.num_yh):
            sum = 0.0
            for i in range(self.num_x):
                sum += self.activation_input[i] * self.weight_in[i][j]

            # 시그모이드와 tanh 중에서 활성화 함수 선택
            self.activation_hidden[j] = tanh(sum, False)

        # 출력층의 활성화 함수
        for k in range(self.num_yo):
            sum = 0.0
            for j in range(self.num_yh):
                sum += self.activation_hidden[j] + self.weight_out[j][k]

            # 시그모이드와 tanh중에서 활성화 함수 선택
            self.activation_out[k] = tanh(sum, False)

        return self.activation_out[:]

    def backPropagate(self, targets):
        output_deltas = [0.0] * self.num_yo
        # 델타 출력 계산
        for k in range(self.num_yo):
            error = targets[k] - self.activation_out[k]
            # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            output_deltas[k] = tanh(self.activation_out[k], True) * error
        
        # 은닉 노드의 오차 함수
        hidden_deltas = [0.0] * self.num_yh
        for j in range(self.num_yh):
            error = 0.0
            for k in range(self.num_yo):
                error += output_deltas[k] * self.weight_out[j][k]
            # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            hidden_deltas[j] = tanh(self.activation_hidden[j], True) * error
        
        # 출력 가중치 업데이트
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                gradient = output_deltas[k] * self.activation_hidden[j]
                v = mo * self.gradient_out[j][k] - lr * gradient
                self.weight_out[j][k] += v
                self.gradient_out[j][k] = gradient

        # 입력 가중치 업데이트
        for i in range(self.num_x):
            for j in range(self.num_yh):
                gradient = hidden_deltas[j] * self.activation_input[i]
                v = mo * self.gradient_in[i][j] - lr * gradient
                self.weight_in[i][j] += v
                self.gradient_in[i][j] = gradient

        # 오차 계산
        error = 0.0
        for k in range(len(targets)):
            error += 0.5 * (targets[k] - self.activation_out[k]) ** 2
        return error

    def train(self, patterns):
        for i in range(itertations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                self.backPropagate(targets)
                error += sum([0.5 * (targets[i] - self.activation_out[i]) ** 2 for i in range(len(targets))])
            if i % 500 == 0:
                print('error: %-.5f' % error)

    # 결과값 출력
    def result(self, patterns):
        for p in patterns:
            print('input: %s, output: %s' % (p[0], self.update(p[0])))

In [30]:
n = NeuralNetwork(2, 2, 1)
n.train(data)
n.result(data)

error: 0.96934
error: 4.99998
error: 4.99998
error: 4.99998
error: 4.99998
error: 4.99998
error: 4.99998
error: 4.99998
error: 4.99998
error: 4.99998
input: [0, 0], output: [np.float64(-0.9999958738557554)]
input: [0, 1], output: [np.float64(-0.9999958738557554)]
input: [1, 0], output: [np.float64(-0.9999958738557554)]
input: [1, 1], output: [np.float64(-0.9999958738557554)]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np

Data_set = np.loadtxt('./data/ThoraricSurgery3.csv', delimiter=',')
X = Data_set[:, 0:16]
y = Data_set[:, 16]

model = Sequential()
model.add(Dense(30, input_dim=16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X, y, epochs=5, batch_size=16)

Epoch 1/5


/Users/junochoi/Documents/University/1학년_2학기/인공신경망/new_venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - accuracy: 0.8511 - loss: 0.5019 
Epoch 2/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.8511 - loss: 0.4980
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.8511 - loss: 0.4867
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - accuracy: 0.8511 - loss: 0.4608
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.8511 - loss: 0.4681
